# CRF

Conditional Random Field

# Vocabulary

In [1]:
Y = ['DT', 'N', 'V']
Y += ['S'] # add in the start token

Y

['DT', 'N', 'V', 'S']

# Tokens

In [2]:
X = ['barks', 'dog', 'the', 'cat', 'meows']

X

['barks', 'dog', 'the', 'cat', 'meows']

# Vectorizers

In [3]:
vy = dict(zip(Y, range(len(Y))))
vx = dict(zip(X, range(len(X))))
vy_ = dict(zip(range(len(Y)), Y))
vx_ = dict(zip(range(len(X)), X))

vy, vx, vy_, vx_

({'DT': 0, 'N': 1, 'V': 2, 'S': 3},
 {'barks': 0, 'dog': 1, 'the': 2, 'cat': 3, 'meows': 4},
 {0: 'DT', 1: 'N', 2: 'V', 3: 'S'},
 {0: 'barks', 1: 'dog', 2: 'the', 3: 'cat', 4: 'meows'})

# Parameters

In [4]:
import numpy as np

Λ = np.random.rand(len(Y), len(Y), len(X))
Λ

array([[[0.69572436, 0.78344193, 0.72009733, 0.8387908 , 0.55442844],
        [0.15617419, 0.61598277, 0.58447397, 0.29841022, 0.80773356],
        [0.45377067, 0.2500105 , 0.3515582 , 0.56109327, 0.20888182],
        [0.86570667, 0.68282748, 0.63333635, 0.14406861, 0.93078327]],

       [[0.97482286, 0.99984454, 0.36385121, 0.71201251, 0.82024602],
        [0.42094199, 0.86725683, 0.79892807, 0.91281138, 0.78052751],
        [0.30840403, 0.83484733, 0.89057677, 0.89243478, 0.80557992],
        [0.40516842, 0.41742901, 0.67848145, 0.55975774, 0.71509369]],

       [[0.55053994, 0.03228043, 0.79713854, 0.35161486, 0.47918231],
        [0.75705646, 0.12374682, 0.40514984, 0.06169626, 0.81922663],
        [0.64233324, 0.30327491, 0.08589306, 0.87952572, 0.9900504 ],
        [0.65612408, 0.55687968, 0.31761705, 0.31239793, 0.18561857]],

       [[0.46835952, 0.20157274, 0.66319877, 0.95389714, 0.15041852],
        [0.91241112, 0.79917948, 0.92988261, 0.11633019, 0.58308228],
        [0.541

# Sample Input

In [5]:
y = ['DT', 'N', 'V']

x = ['the', 'dog', 'barks']

print(y)
print(x)

['DT', 'N', 'V']
['the', 'dog', 'barks']


# Unnormalized predict

Compute $\text{score}(\mathbf{y}, \mathbf{x})$

In [6]:
def predict_score(y, x):
    y_ = ['S'] + y
    x_ = [None] + x
    n = len(x_)
    score = 0
    for i in range(1, n):
        λ = Λ[vy[y_[i-1]], vy[y_[i]], vx[x_[i]]]
        score += np.exp(λ)
    return score

predict_score(y, x)

5.153717337672218

# Normalized predict

Compute $p(\mathbf{y} | \mathbf{x}) = \frac{\displaystyle \text{score}(\mathbf{y}, \mathbf{x})}{\displaystyle \sum_{\mathbf{y}'} \text{score}(\mathbf{y}, \mathbf{x})}$

# Initialize DP Table

In [7]:
import pandas as pd

dp = np.zeros([len(y), len(x)])

df = pd.DataFrame(dp, index=y, columns=x)
df

,the,dog,barks
DT,0.0,0.0,0.0
N,0.0,0.0,0.0
V,0.0,0.0,0.0


# Initialize first column

By hand ✍️

In [8]:
dp[0, 0] = Λ[vy['S'], vy['DT'], vx['the']]
dp[1, 0] = Λ[vy['S'], vy['N'], vx['the']]
dp[2, 0] = Λ[vy['S'], vy['N'], vx['the']]

df = pd.DataFrame(dp, index=y, columns=x)
df

,the,dog,barks
DT,0.663199,0.0,0.0
N,0.929883,0.0,0.0
V,0.929883,0.0,0.0


# Fill in the rest of the table

By hand ✍️

In [9]:
dp[0, 1] = dp[0, 0]*Λ[vy['DT'], vy['DT'], vx['dog']] + \
           dp[1, 0]*Λ[vy['N'], vy['DT'], vx['dog']] + \
           dp[2, 0]*Λ[vy['V'], vy['DT'], vx['dog']]

dp[1, 1] = dp[0, 0]*Λ[vy['DT'], vy['N'], vx['dog']] + \
           dp[1, 0]*Λ[vy['N'], vy['N'], vx['dog']] + \
           dp[2, 0]*Λ[vy['V'], vy['N'], vx['dog']]

dp[2, 1] = dp[0, 0]*Λ[vy['DT'], vy['N'], vx['dog']] + \
           dp[1, 0]*Λ[vy['N'], vy['N'], vx['dog']] + \
           dp[2, 0]*Λ[vy['V'], vy['N'], vx['dog']]

df = pd.DataFrame(dp, index=y, columns=x)
df

,the,dog,barks
DT,0.663199,1.479333,0.0
N,0.929883,1.330036,0.0
V,0.929883,1.330036,0.0


In [10]:
dp[0, 2] = dp[0, 1]*Λ[vy['DT'], vy['DT'], vx['dog']] + \
           dp[1, 1]*Λ[vy['N'], vy['DT'], vx['dog']] + \
           dp[2, 1]*Λ[vy['V'], vy['DT'], vx['dog']]

dp[1, 2] = dp[0, 1]*Λ[vy['DT'], vy['N'], vx['dog']] + \
           dp[1, 1]*Λ[vy['N'], vy['N'], vx['dog']] + \
           dp[2, 1]*Λ[vy['V'], vy['N'], vx['dog']]

dp[2, 2] = dp[0, 1]*Λ[vy['DT'], vy['N'], vx['dog']] + \
           dp[1, 1]*Λ[vy['N'], vy['N'], vx['dog']] + \
           dp[2, 1]*Λ[vy['V'], vy['N'], vx['dog']]

df = pd.DataFrame(dp, index=y, columns=x)
df

,the,dog,barks
DT,0.663199,1.479333,2.531735
N,0.929883,1.330036,2.229314
V,0.929883,1.330036,2.229314


In [11]:
def predict_probability(y, x):
    numerator = predict_score(y, x)
    denominator = dp[:, -1].sum()
    return numerator / denominator

predict_probability(y, x)

0.7372603322796317